In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import os
import csv
import re
import traceback
import datetime
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options



In [ ]:
def human_login(elem, login_info):
    info_len = len(login_info)
    if info_len < 1:
        print("Invalid LoginInfo")
        return
    i = 0
    while i < info_len:
        choice = random.randint(1, 5)
        if choice == 5 and random.randint(0, 5) > 3:
            if i == 0:
                continue
            else:
                elem.send_keys(Keys.BACKSPACE)
                i -= 1
        else:
            elem.send_keys(login_info[i])
            i += 1
            time.sleep(0.1 + choice * 0.02)


options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])

options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

In [ ]:
# 进入淘宝官网
driver.get('https://www.taobao.com/')
driver.maximize_window()
# 定位到登录键
elem = driver.find_element("class name", "h")
elem.click()
time.sleep(2)
# 定位元素到账号一栏
elem = driver.find_element("id", "fm-login-id")
# 开始“类人”输入账号
human_login(elem, "#")
# 睡一会
time.sleep(0.5)
# 定位元素到密码一栏
elem = driver.find_element("id", "fm-login-password")
# 开始“类人”输入密码
human_login(elem, "#")
# 按下 enter 键
elem.send_keys(Keys.RETURN)
# 再睡会
# 这里睡的时间那么长，是为了防止出现验证界面（系统自己检测成功后会自动跳转）
time.sleep(3)
# 这里的定位很简单

In [2]:
def save_csv(data_dic,file_name,i):
    file_exists = os.path.exists(file_name)

    with open(file_name, 'a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data_dic.keys())

        # 如果文件不存在，写入表头
        if not file_exists:
            writer.writeheader()

        # 写入数据
        writer.writerow(data_dic)
    print(i,'turns:''success')

In [3]:
def scroll(driver): 
    total_height = driver.execute_script("return document.body.scrollHeight")

    # 计算位置
    scroll_to = total_height * 0.20

    # 每次滚动增加的高度
    increment = 200  # 您可以根据需要调整这个值

    # 当前滚动位置
    current_scroll_position = 0

    while current_scroll_position < scroll_to:
        # 增加滚动位置
        current_scroll_position += increment

        # 滚动到新位置
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

        # 短暂延时
        time.sleep(0.1)  # 每次滚动后的等待时间，可以根据需要调整

        # 防止滚动超出目标位置
        if current_scroll_position > scroll_to:
            driver.execute_script(f"window.scrollTo(0, {scroll_to});")
            break
    time.sleep(2)

In [4]:
df=pd.read_csv('cleanedData.csv')
urls=df['product_link']

In [38]:
continous_fail=0
for i in complete:
    if continous_fail>=5:
        #time.sleep(300)
        print("滑块")
        break
    driver.get(urls[i])
    scroll(driver)
    time.sleep(3)
    try:
        time.sleep(3)
        '''
        //*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[1]/div[2]
        //*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[2]/div[2]
        '''
        #店铺评分

        product_discrp=driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[1]/div[2]').text
        mkt_service=driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[2]/div[2]').text
        mailing=driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[3]/div[2]').text

        '''
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[1]/div/ul/li[1]/img
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[1]/div/ul/li[2]/img
        '''

        #商品链接

        gallery=driver.find_element(By.XPATH,'//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[1]/div/ul')
        li_elements = gallery.find_elements(By.TAG_NAME, 'li')

        src_list = []

        # 遍历每个 'li' 元素，找到其中的 'img' 标签并获取 'src' 属性
        for li in li_elements:
            img_element = li.find_element(By.TAG_NAME, 'img')
            src = img_element.get_attribute('src')
            src_list.append(src)


        #商品详情
        '''
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]

        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]/div[1]
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]/div[2]
        #<div class="Attrs--attrSection--2_G8xGa"><span class="Attrs--attr--33ShB6X" title="品牌：顶峰保罗">品牌：顶峰保罗</span><span class="Attrs--attr--33ShB6X" title="尺码：S M L XL 2XL 3XL">尺码：S M L XL 2XL 3XL</span><span class="Attrs--attr--33ShB6X" title="面料分类：其他">面料分类：其他</span></div>
        '''
        product_details = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]') 
        div_html = product_details.get_attribute('innerHTML')

        # 使用 BeautifulSoup 解析 HTML
        soup = BeautifulSoup(div_html, 'html.parser') if div_html!="" else BeautifulSoup(driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[5]').get_attribute('innerHTML') , 'html.parser')


        # 初始化一个字典来存储商品信息
        product_detail = {}

        # 遍历所有的 span 元素
        for span in soup.find_all('span', class_='Attrs--attr--33ShB6X'):
            # 提取每个 span 的 title 属性
            title_text = span.get('title')
            if title_text:
                # 分割 title 文本来获取键和值
                key, value = re.split(r'：', title_text, maxsplit=1)
                product_detail[key] = value
        pro_info={"SN":i,"product_discrip":product_discrp,'mkt_service':mkt_service,'mailing':mailing,'src_list':src_list,'product_detail':product_detail}

        save_csv(pro_info,'product_info4.csv',i)
        continous_fail=0
        time.sleep(2)
        
        ##########################################
    except NoSuchElementException:
        try:
            time.sleep(3)
            '''           
            #描述
            //*[@id="header-content"]/div[2]/div[2]/ul/li[1]/span[2]/em
            #服务
            //*[@id="header-content"]/div[2]/div[2]/ul/li[2]/span[2]/em
            #物流
            //*[@id="header-content"]/div[2]/div[2]/ul/li[3]/span[2]/em
            '''
            #店铺评分
            shops_scores=driver.find_element(By.XPATH, '//*[@id="header-content"]/div[2]/div[2]/ul')
            shop_li_elements = shops_scores.find_elements(By.TAG_NAME, 'li')

            product_discrp = shop_li_elements[0].find_element(By.XPATH, './span[2]/em').text

            mkt_service = shop_li_elements[1].find_element(By.XPATH, './span[2]/em').text

            mailing = shop_li_elements[2].find_element(By.XPATH, './span[2]/em').text



            #图片
            '''
            #<img data-src="//gd1.alicdn.com/imgextra/i3/0/O1CN0111PCrm20ToKnPvRWQ_!!0-item_pic.jpg_50x50.jpg" src="//gd1.alicdn.com/imgextra/i3/0/O1CN0111PCrm20ToKnPvRWQ_!!0-item_pic.jpg_50x50.jpg_.webp">
            //*[@id="J_UlThumb"]/li[1]/div/a/img
            //*[@id="J_UlThumb"]/li[2]/div/a/img
            '''

            gallery=driver.find_element(By.XPATH,'//*[@id="J_UlThumb"]')
            li_elements = gallery.find_elements(By.TAG_NAME, 'li')

            src_list = []

            # 遍历每个 'li' 元素，找到其中的 'img' 标签并获取 'src' 属性
            for li in li_elements:
                img_element = li.find_element(By.TAG_NAME, 'img')
                src = img_element.get_attribute('src')
                src_list.append(src)

            #商品详情
            '''
            #<li title="棉致">品牌:&nbsp;棉致</li>
            //*[@id="attributes"]/ul
            '''
            ul_element = driver.find_element(By.XPATH, '//*[@id="attributes"]/ul')

            # 初始化一个字典来存储产品详情
            product_detail = {}

            # 遍历 'ul' 元素下的所有 'li' 元素
            for li in ul_element.find_elements(By.TAG_NAME, 'li'):
                # 获取 'li' 元素的 'title' 属性
                key = li.get_attribute('title')

                # 获取 'li' 元素的文本，并去除冒号前的文本（如“品牌:&nbsp;”）
                text = li.text
                value = re.split(r':\s*&nbsp;', text, maxsplit=1)[1] if ':&nbsp;' in text else text

                product_detail[key] = value


            pro_info={"SN":i,"product_discrip":product_discrp,'mkt_service':mkt_service,'mailing':mailing,'src_list':src_list,'product_detail':product_detail}

            save_csv(pro_info,'product_info4.csv',i)
            continous_fail=0

     
        except Exception as e:
            # 获取完整的错误信息

            error_info = traceback.format_exc()

            # 获取当前时间
            current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # 将错误信息、当前时间和 URL 写入文件
            with open('errors4.csv', 'a') as file:
                file.write(f"{i},{current_time},{urls[i]},{error_info.replace(',', ';')}\n")
            print(i,' turns:failed')
            continous_fail+=1
            time.sleep(3)

 
        

4217 turns:success
4218 turns:success
4219 turns:success
4220 turns:success
4221 turns:success
4222  turns:failed
4223 turns:success
4224 turns:success
4225 turns:success
4226  turns:failed
4227 turns:success
4228 turns:success
4229 turns:success
4230  turns:failed
4231 turns:success
4232 turns:success
4233 turns:success
4234 turns:success
4235 turns:success
4236 turns:success
4237  turns:failed
4238 turns:success
4239 turns:success
4240 turns:success
4241 turns:success
4242 turns:success
4243 turns:success
4244 turns:success
4245 turns:success
4246 turns:success
4247 turns:success
4248 turns:success
4249 turns:success
4250 turns:success
4251 turns:success
4252 turns:success
4253  turns:failed
4254 turns:success
4255 turns:success
4256 turns:success
4257 turns:success
4258 turns:success
4259 turns:success
4260 turns:success
4261 turns:success
4262 turns:success
4263 turns:success
4264 turns:success
4265 turns:success
4266 turns:success
4267 turns:success
4268 turns:success
4269  turns:

In [37]:
finished=pd.read_csv('product_info4.csv')['SN'].to_list()
complete=[i for i in list(range(3000,len(urls))) if i not in finished and i >4216
         ]
complete


[4217,
 4218,
 4219,
 4220,
 4221,
 4222,
 4223,
 4224,
 4225,
 4226,
 4227,
 4228,
 4229,
 4230,
 4231,
 4232,
 4233,
 4234,
 4235,
 4236,
 4237,
 4238,
 4239,
 4240,
 4241,
 4242,
 4243,
 4244,
 4245,
 4246,
 4247,
 4248,
 4249,
 4250,
 4251,
 4252,
 4253,
 4254,
 4255,
 4256,
 4257,
 4258,
 4259,
 4260,
 4261,
 4262,
 4263,
 4264,
 4265,
 4266,
 4267,
 4268,
 4269,
 4270,
 4271,
 4272,
 4273,
 4274,
 4275,
 4276,
 4277,
 4278,
 4279,
 4280,
 4281,
 4282,
 4283,
 4284,
 4285,
 4286,
 4287,
 4288,
 4289,
 4290,
 4291,
 4292,
 4293,
 4294,
 4295,
 4296,
 4297,
 4298,
 4299,
 4300,
 4301,
 4302,
 4303,
 4304,
 4305,
 4306,
 4307,
 4308,
 4309,
 4310,
 4311,
 4312,
 4313,
 4314,
 4315,
 4316,
 4317,
 4318,
 4319,
 4320,
 4321,
 4322,
 4323,
 4324,
 4325,
 4326,
 4327,
 4328,
 4329,
 4330,
 4331,
 4332,
 4333,
 4334,
 4335,
 4336,
 4337,
 4338,
 4339,
 4340,
 4341,
 4342,
 4343,
 4344,
 4345,
 4346,
 4347,
 4348,
 4349,
 4350,
 4351,
 4352,
 4353,
 4354,
 4355,
 4356,
 4357,
 4358,
 4359,

In [149]:
driver.get(urls[3775])


In [54]:
len(urls)

4394

拼接补充

In [39]:
df1=pd.read_csv('product_info1.csv')
df2=pd.read_csv('product_info2.csv')
df3=pd.read_csv('product_info3.csv')
df4=pd.read_csv('product_info4.csv')
f=pd.concat([df1, df2, df3, df4], axis=0)
f = f.drop_duplicates(subset=['SN'])
f.head()

,SN,product_discrip,mkt_service,mailing,src_list,product_detail
0,2,4.6,4.7,4.7,['https://gw.alicdn.com/imgextra/i1/3055237276...,"{'品牌': 'VUUG', '尺码': 'M L XL 2XL 3XL 4XL', '面料..."
1,3,4.7,4.7,4.7,['https://gw.alicdn.com/imgextra/i3/2211499841...,"{'品牌': 'Antin Soul Studios', '尺码': 'M L XL 2XL..."
2,4,4.9,4.9,4.9,['https://gd4.alicdn.com/imgextra/i3/0/O1CN011...,"{'棉致': '品牌: 棉致', 'M L XL 2XL 3XL': '尺码: M L XL..."
3,5,4.7,4.7,4.7,['https://gw.alicdn.com/imgextra/O1CN015CATd61...,"{'品牌': '贵人鸟', '功能性': '可脱卸内胆', '尺码': 'M L XL 2X..."
4,6,4.8,4.9,4.9,['https://gw.alicdn.com/imgextra/i4/6000000000...,"{'品牌': 'Romon/罗蒙', '尺码': 'M L XL 2XL 3XL 4XL',..."


In [47]:
f.query('product_detail == "{}"')

,SN,product_discrip,mkt_service,mailing,src_list,product_detail
8,11,4.7,4.7,4.8,['https://gw.alicdn.com/imgextra/i4/2380441317...,{}
15,22,4.7,4.7,4.8,['https://gw.alicdn.com/imgextra/i4/1819994465...,{}
18,25,4.7,4.7,4.8,['https://gw.alicdn.com/imgextra/i2/1819994465...,{}
19,26,4.7,4.8,4.8,['https://gw.alicdn.com/imgextra/i3/363607599/...,{}
20,30,4.7,4.7,4.8,['https://gw.alicdn.com/imgextra/i1/2380441317...,{}
...,...,...,...,...,...,...
814,2988,4.8,4.8,4.8,['https://gw.alicdn.com/imgextra/i2/2201209761...,{}
819,2993,4.7,4.7,4.7,['https://gw.alicdn.com/imgextra/O1CN01iuzUKJ1...,{}
821,2995,4.7,4.7,4.7,['https://gw.alicdn.com/imgextra/i1/925814798/...,{}
823,2998,4.7,4.7,4.8,['https://gw.alicdn.com/imgextra/O1CNA1liG1vu2...,{}


In [5]:
#result = pd.merge(df,f, left_on='num', right_on='SN', how='left')
result=pd.read_csv('final.csv')

In [16]:
cc=result.query('product_detail == "{}"')
cc

,Unnamed: 0,num,name,price,sales,shop_name,shop_link,product_link,image_link,shipping_insurance,...,city,sales_category,merged_sales_category,words,SN,product_discrip,mkt_service,mailing,src_list,product_detail
2966,2966,2966,HM男装羽绒服2023冬季新款双面穿休闲柔软简约保暖外套1200589,499,68,HM官方旗舰店,https://store.taobao.com/shop/view_shop.htm?ap...,https://detail.tmall.com/item.htm?id=742834090...,https://g-search3.alicdn.com/img/bao/uploaded/...,False,...,苏州,<100,small,"['HM', '男装', '羽绒服', '2023', '冬季', '新款', '双面', ...",2966.0,4.7,4.8,4.8,['https://gw.alicdn.com/imgextra/i1/3676232520...,{}


In [17]:
continous_fail=0
for index, row in cc.iterrows():

    if continous_fail>=5:
        #time.sleep(300)
        print("滑块")
        break
    driver.get(row['product_link'])
    scroll(driver)
    time.sleep(3)

    try:
        time.sleep(3)


        #商品详情

        product_details = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]') 
        div_html = product_details.get_attribute('innerHTML')

        # 使用 BeautifulSoup 解析 HTML
        soup = BeautifulSoup(div_html, 'html.parser') if div_html!="" else BeautifulSoup(driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[5]').get_attribute('innerHTML') , 'html.parser')


        # 初始化一个字典来存储商品信息
        product_detail = {}

        # 遍历所有的 span 元素
        for span in soup.find_all('span', class_='Attrs--attr--33ShB6X'):
            # 提取每个 span 的 title 属性
            title_text = span.get('title')
            if title_text:
                # 分割 title 文本来获取键和值
                key, value = re.split(r'：', title_text, maxsplit=1)
                product_detail[key] = value
                
        result.at[index, 'product_detail'] = product_detail
        print(index,"success",':',len(product_detail))
        continous_fail=0
        time.sleep(2)
        
        ##########################################
    except NoSuchElementException:
        try:
            time.sleep(3)

            #商品详情

            ul_element = driver.find_element(By.XPATH, '//*[@id="attributes"]/ul')

            # 初始化一个字典来存储产品详情
            product_detail = {}

            # 遍历 'ul' 元素下的所有 'li' 元素
            for li in ul_element.find_elements(By.TAG_NAME, 'li'):
                # 获取 'li' 元素的 'title' 属性
                key = li.get_attribute('title')

                # 获取 'li' 元素的文本，并去除冒号前的文本（如“品牌:&nbsp;”）
                text = li.text
                value = re.split(r':\s*&nbsp;', text, maxsplit=1)[1] if ':&nbsp;' in text else text

                product_detail[key] = value
            # fill
            
            continous_fail=0
            result.at[index, 'product_detail'] = product_detail
            print(index,"success",product_detail,':',len(product_detail))

     
        except Exception as e:
            # 获取完整的错误信息

            error_info = traceback.format_exc()

            # 获取当前时间
            current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # 将错误信息、当前时间和 URL 写入文件
            with open('errors_complete.csv', 'a') as file:
                file.write(f"{index},{current_time},{row['product_link']},{error_info.replace(',', ';')}\n")
            print(index,' turns:failed')
            continous_fail+=1
            time.sleep(3)

 
        

2966 success : 12


In [18]:
result.to_csv('final.csv',encoding='utf_8_sig')